In [26]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow as tf

In [41]:
T1 = Sequential()

T1.add(layers.Flatten(input_shape=(32,32)))
T1.add(layers.Dense(units=1024, activation="tanh"))
T1.add(layers.Dense(units=512, activation="sigmoid"))
T1.add(layers.Dense(units=100, activation="softmax"))

# Race classifier
T1.add(layers.Dense(units=4, activation="softmax"))

# Gender classifier
#T1.add(layers.Dense(units=2, activation="softmax"))

# Age classifier
#T1.add(layers.Dense(units=10, activation="softmax"))

print(T1.summary())

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_12 (Flatten)         (None, 1024)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_21 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_22 (Dense)             (None, 100)               51300     
_________________________________________________________________
dense_23 (Dense)             (None, 4)                 404       
Total params: 1,626,104
Trainable params: 1,626,104
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
T2 = Sequential()

T2.add(layers.Conv2D(filters=40, kernel_size=(5,5), activation="relu", strides=(1,1), padding="same", input_shape=(32,32,1)))
T2.add(layers.MaxPool2D(pool_size=(2,2)))
T2.add(layers.Flatten())
T2.add(layers.Dense(units=100, activation="relu"))

# Race classifier
#T2.add(layers.Dense(units=4, activation="softmax"))

# Gender classifier
#T2.add(layers.Dense(units=2, activation="softmax"))

# Age classifier
T2.add(layers.Dense(units=10, activation="softmax"))

print(T2.summary())

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 32, 32, 40)        1040      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 40)        0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 10240)             0         
_________________________________________________________________
dense_29 (Dense)             (None, 100)               1024100   
_________________________________________________________________
dense_30 (Dense)             (None, 10)                1010      
Total params: 1,026,150
Trainable params: 1,026,150
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
T3 = Sequential()

# NEED TO CHOOSE PARAMS

T2.add(layers.Conv2D(filters=40, kernel_size=(5,5), activation="relu", strides=(1,1), padding="same", input_shape=(32,32,1)))
T2.add(layers.MaxPool2D(pool_size=(2,2)))
T2.add(layers.Flatten())
T2.add(layers.Dense(units=100, activation="relu"))

# Race classifier
#T2.add(layers.Dense(units=4, activation="softmax"))

# Gender classifier
#T2.add(layers.Dense(units=2, activation="softmax"))

# Age classifier
T2.add(layers.Dense(units=10, activation="softmax"))

In [5]:
T4 = Sequential()

In [6]:
T5 = Sequential()